In [1]:
# Path setup / Imports
import sys
repo_root = "../"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
    
import yaml
import os
from transition import RNNTransition

In [2]:
# Get config
yaml_path = "config/training/train_rnn_smiles.yaml"
with open(os.path.join(repo_root, yaml_path)) as f:
    conf = yaml.safe_load(f)
output_dir = os.path.join(repo_root, conf.pop("output_dir"))

# Start training
model, best_state_dict, lang = RNNTransition.train_rnn_from_conf(conf)

Is CUDA available: True


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/224510 [00:00<?, ? examples/s]

Map:   0%|          | 0/24946 [00:00<?, ? examples/s]

Epoch 1: 100%|██████████| 439/439 [00:11<00:00, 37.28it/s]


...

[98] train_loss: 0.6080  val_loss: 0.6062


Epoch 99: 100%|██████████| 439/439 [00:11<00:00, 37.55it/s]


[99] train_loss: 0.6084  val_loss: 0.6058


Epoch 100: 100%|██████████| 439/439 [00:11<00:00, 38.12it/s]


[100] train_loss: 0.6092  val_loss: 0.6064


In [3]:
# Save models
model.save(os.path.join(output_dir, "last"))
model.load_state_dict(best_state_dict)
model.save(os.path.join(output_dir, "best"))

In [4]:
# Save yaml
import shutil
src = os.path.join(repo_root, yaml_path)
dst = os.path.join(output_dir, "config.yaml")
shutil.copy(src, dst)

# Save lang
if lang.__class__.__name__ == "HELM": # make and save the converter if the language is HELM
    lib_files = ["chembl_35_monomer_library.xml", "HELMCoreLibrary.json", "monomerLib2.0.json"]
    lang.load_monomer_library(*[repo_root + f"data/helm/library/{name}" for name in lib_files], culling=True)

lang.save(os.path.join(output_dir, "language.lang"))